In [1]:
import numpy as np

import pandas as pd
from pandas import Series,DataFrame
from bokeh.layouts import gridplot, row, column

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure

from sklearn.datasets import load_boston

# For very simple visualizations (without too much interaction, output_notebook works too, which we enable here)
output_notebook()

Loading BokehJS ...

In [64]:
# Load dataset
df18 = pd.read_csv('../Data_csv/data_2018.csv')
for columnname in df18:
    print(columnname)
df18['stats_pc_intl_students'] = df18['stats_pc_intl_students'].map(lambda x: x.rstrip('%'))
df18 = df18[:199]
        
rank = df18['rank']
location = df18['location']
percentage_male = df18['percentage_male']
stats_number_students = df18['stats_number_students']
stats_student_staff_ratio = df18['stats_student_staff_ratio']

columnlist = [location, percentage_male, stats_number_students, stats_student_staff_ratio, rank]

Unnamed: 0
location
name
nid
rank
rank_order
scores_citations
scores_citations_rank
scores_industry_income
scores_industry_income_rank
scores_international_outlook
scores_international_outlook_rank
scores_overall
scores_research
scores_research_rank
scores_teaching
scores_teaching_rank
percentage_male
stats_number_students
stats_pc_intl_students
stats_student_staff_ratio
subjects_offered
url
male_students


In [65]:
# You can see what these names mean in the description that we printed at the start
# print(df18.feature_names)

figures = [figure() for _ in range(24)]
for fig, column in zip(figures, df18):
    # Create a scatter-plot
    fig.scatter(df18[column], df18["rank"])
    
    ## Add some axis information
    fig.xaxis.axis_label = 'rank'
    fig.yaxis.axis_label = column

    
show(gridplot(figures, ncols=2, plot_width=400, plot_height=250, toolbar_location=None))

In [63]:
from sklearn.metrics import mean_squared_error
msecolumn = []

# calculate all mse's
for column in df18:
    try:
        mse = mean_squared_error(np.array(df18[column]),df18['rank_order'])
        print(mse)
        msecolumn.append((mse,column))
    except ValueError:
        print(column + ' ..whoeps')
        
print()
print(mselist)

# find lowest
sorted(msecolumn)

# r = np.array(df18['rank_order'])
# y = np.array(df18['rank'])
# mse = mean_squared_error(r, y)
# print(mse)

1.0
location ..whoeps
name ..whoeps
76851659974.4
rank ..whoeps
0.0
372150.901659
15337.0480508
362819.370426
114134.846782
366676.71447
64983.2112421
scores_overall ..whoeps
389013.032566
27847.2801451
381111.497162
45720.3553944
352063.821396
3907841380.93
397158.900272
385231.644025
subjects_offered ..whoeps
url ..whoeps
male_students ..whoeps

[1.0, 76851659974.449677, 0.0, 372150.90165911155, 15337.048050770625, 362819.37042611057, 114134.84678150498, 366676.71446962829, 64983.211242067089, 389013.03256572981, 27847.280145058929, 381111.49716228468, 45720.355394378967, 352063.82139619219, 3907841380.9347234, 385231.64402538532]


[(0.0, 'rank_order'),
 (1.0, 'Unnamed: 0'),
 (15337.048050770625, 'scores_citations_rank'),
 (27847.280145058929, 'scores_research_rank'),
 (45720.355394378967, 'scores_teaching_rank'),
 (64983.211242067089, 'scores_international_outlook_rank'),
 (114134.84678150498, 'scores_industry_income_rank'),
 (352063.82139619219, 'percentage_male'),
 (362819.37042611057, 'scores_industry_income'),
 (366676.71446962829, 'scores_international_outlook'),
 (372150.90165911155, 'scores_citations'),
 (381111.49716228468, 'scores_teaching'),
 (385231.64402538532, 'stats_student_staff_ratio'),
 (389013.03256572981, 'scores_research'),
 (397158.90027198551, 'stats_pc_intl_students'),
 (3907841380.9347234, 'stats_number_students'),
 (76851659974.449677, 'nid')]

In [67]:
from sklearn.metrics import mean_squared_error
msecolumn = []

# calculate all mse's
for column in df18:
    try:
        mse = mean_squared_error(np.array(df18[column]),df18['rank_order'])
#         print(mse)
        msecolumn.append((mse,column))
    except ValueError:
        print(column + ' ..whoeps')
        

# find lowest
sorted(msecolumn)

# r = np.array(df18['rank_order'])
# y = np.array(df18['rank'])
# mse = mean_squared_error(r, y)
# print(mse)

location ..whoeps
name ..whoeps
rank ..whoeps
subjects_offered ..whoeps
url ..whoeps
male_students ..whoeps


[(0.0, 'rank_order'),
 (1.0, 'Unnamed: 0'),
 (2394.221105527638, 'scores_research_rank'),
 (4241.5283919597987, 'scores_citations'),
 (4776.3218090452265, 'scores_international_outlook'),
 (5866.4549748743721, 'scores_overall'),
 (5931.3567839195975, 'scores_teaching_rank'),
 (5953.0525628140704, 'scores_industry_income'),
 (6025.4824120603016, 'percentage_male'),
 (7352.1187939698502, 'scores_teaching'),
 (7386.4450251256276, 'scores_research'),
 (9801.3316582914576, 'stats_pc_intl_students'),
 (10045.962462311556, 'stats_student_staff_ratio'),
 (15787.27135678392, 'scores_citations_rank'),
 (66112.819095477389, 'scores_international_outlook_rank'),
 (112102.09547738693, 'scores_industry_income_rank'),
 (784003101.42211056, 'stats_number_students'),
 (4181784600.6582913, 'nid')]

In [71]:
# Import for Linear Regression
import sklearn
from sklearn.linear_model import LinearRegression

# Create a LinearRegression Object
lreg = LinearRegression()

In [93]:
# Data Columns
df18new = df18.drop('url', axis=1)
df18new = df18new.drop('subjects_offered', axis=1)
# df18new = df18new.drop('rank_order', axis=1)
df18new = df18new.drop('name', axis=1)
df18new = df18new.drop('location', axis=1)
df18new = df18new.drop('male_students', axis=1)
df18new = df18new.drop('nid', axis=1)
df18new = df18new.drop('scores_industry_income', axis=1)
df18new = df18new.drop('scores_international_outlook', axis=1)
df18new = df18new.drop('scores_research', axis=1)
df18new = df18new.drop('scores_teaching', axis=1)
df18new = df18new.drop('scores_citations', axis=1)
df18new = df18new.drop('Unnamed: 0', axis=1)
df18new = df18new.drop('rank', axis=1)
df18new = df18new.drop('scores_overall', axis=1)

X_multi = df18new.drop('rank_order',axis=1)

for name in df18new:
    print(name)

# Targets
Y_target = df18new['rank_order']

# Implement Linear Regression
lreg.fit(X_multi,Y_target)

rank_order
scores_citations_rank
scores_industry_income_rank
scores_international_outlook_rank
scores_research_rank
scores_teaching_rank
percentage_male
stats_number_students
stats_pc_intl_students
stats_student_staff_ratio


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [94]:
# What is our constant? (b in univariate regression)
print(' The estimated intercept coefficient is {0:.2f}'.format(lreg.intercept_))

print(' The number of coefficients used was {0:d}'.format(len(lreg.coef_)))

 The estimated intercept coefficient is -2.21
 The number of coefficients used was 9


In [95]:
coeff_df = DataFrame(df18new.drop('rank_order',axis=1).columns)
coeff_df.columns = ['features']

coeff_df["Coefficient Estimate"] = pd.Series(lreg.coef_)

coeff_df.sort_values(by='Coefficient Estimate', ascending=False)

,features,Coefficient Estimate
3,scores_research_rank,0.388338
4,scores_teaching_rank,0.197538
0,scores_citations_rank,0.185523
8,stats_student_staff_ratio,0.097495
5,percentage_male,0.017601
2,scores_international_outlook_rank,0.017065
1,scores_industry_income_rank,0.013970
6,stats_number_students,-0.000020
7,stats_pc_intl_students,-0.387316


In [128]:
X = np.vstack(df18new.scores_research_rank)
X = np.column_stack((X, np.ones(X.shape[0])))
print(X)

[[   1.    1.]
 [   3.    1.]
 [   4.    1.]
 [   5.    1.]
 [   9.    1.]
 [   2.    1.]
 [   6.    1.]
 [  12.    1.]
 [  10.    1.]
 [   8.    1.]
 [  11.    1.]
 [  17.    1.]
 [  16.    1.]
 [  26.    1.]
 [  15.    1.]
 [  13.    1.]
 [  27.    1.]
 [  24.    1.]
 [  19.    1.]
 [  18.    1.]
 [  20.    1.]
 [  14.    1.]
 [  23.    1.]
 [  25.    1.]
 [  43.    1.]
 [  28.    1.]
 [  36.    1.]
 [  34.    1.]
 [  22.    1.]
 [   7.    1.]
 [  29.    1.]
 [  37.    1.]
 [  30.    1.]
 [  40.    1.]
 [  39.    1.]
 [  44.    1.]
 [  31.    1.]
 [  54.    1.]
 [  35.    1.]
 [  33.    1.]
 [  46.    1.]
 [  48.    1.]
 [  38.    1.]
 [  50.    1.]
 [  69.    1.]
 [  21.    1.]
 [  47.    1.]
 [  41.    1.]
 [  51.    1.]
 [  78.    1.]
 [  73.    1.]
 [  65.    1.]
 [  72.    1.]
 [  58.    1.]
 [  56.    1.]
 [  52.    1.]
 [  70.    1.]
 [  59.    1.]
 [  66.    1.]
 [  49.    1.]
 [  62.    1.]
 [  53.    1.]
 [  42.    1.]
 [  96.    1.]
 [  74.    1.]
 [  76.    1.]
 [  61.   

In [129]:
import sklearn.model_selection
# Grab the output and set as X and Y test and train data sets!
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,df18new.rank_order)

# Print shapes of the training and testing data sets
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
print(X_train)

(149, 2) (50, 2) (149,) (50,)
[[   7.    1.]
 [ 119.    1.]
 [ 195.    1.]
 [ 136.    1.]
 [  77.    1.]
 [  91.    1.]
 [  17.    1.]
 [  94.    1.]
 [  59.    1.]
 [  43.    1.]
 [ 116.    1.]
 [ 113.    1.]
 [  48.    1.]
 [   5.    1.]
 [  36.    1.]
 [  33.    1.]
 [  89.    1.]
 [  97.    1.]
 [ 109.    1.]
 [ 263.    1.]
 [ 126.    1.]
 [  53.    1.]
 [ 121.    1.]
 [ 118.    1.]
 [  64.    1.]
 [ 145.    1.]
 [ 124.    1.]
 [  27.    1.]
 [  50.    1.]
 [ 202.    1.]
 [  61.    1.]
 [ 128.    1.]
 [ 117.    1.]
 [  71.    1.]
 [  12.    1.]
 [ 226.    1.]
 [ 115.    1.]
 [ 201.    1.]
 [  58.    1.]
 [ 419.    1.]
 [   9.    1.]
 [  41.    1.]
 [ 141.    1.]
 [ 158.    1.]
 [ 205.    1.]
 [ 198.    1.]
 [ 105.    1.]
 [ 232.    1.]
 [ 184.    1.]
 [  49.    1.]
 [ 103.    1.]
 [   6.    1.]
 [ 345.    1.]
 [  73.    1.]
 [   8.    1.]
 [ 138.    1.]
 [ 156.    1.]
 [  57.    1.]
 [  55.    1.]
 [  80.    1.]
 [ 125.    1.]
 [ 134.    1.]
 [  32.    1.]
 [  62.    1.]
 [  87.   

In [130]:
# Create our regression object
lreg = LinearRegression()

# Once again do a linear regression, except only on the training sets this time
lreg.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [131]:
# Predictions on training and testing sets
pred_train = lreg.predict(X_train)
pred_test = lreg.predict(X_test)

In [136]:
from sklearn.metrics import mean_squared_error
# mse = mean_squared_error(r, y)

print("Fit a model X_train, and calculate MSE with Y_train: {0:.2f}".format(mean_squared_error(Y_train, pred_train)))
    
print("Fit a model X_train, and calculate MSE with X_test and Y_test: {0:.2f}".format(mean_squared_error(Y_test, pred_test)))

Fit a model X_train, and calculate MSE with Y_train: 1013.19
Fit a model X_train, and calculate MSE with X_test and Y_test: 865.64
